# (CTC-17) Projeto de Buscas
---

## Dependências

In [1]:
import pandas as pd
from math import sqrt

## Importando os dados

In [2]:
# Import
australia = pd.read_csv('australia.csv', sep = ",")
# Cleaning
australia = australia[['id', 'city', 'lat', 'lng']]
# Printing
australia

,id,city,lat,lng
0,1,Adelaide,-34.928661,138.598633
1,2,Adelaide River,-13.239430,131.107330
2,3,Albany,-35.003101,117.865952
3,4,Albury,-36.074823,146.924006
4,5,Alice Springs,-23.697479,133.883621
...,...,...,...,...
214,215,Woomera,-31.199810,136.832581
215,216,Yamba,-29.435450,153.360306
216,217,Yeppoon,-23.126829,150.744064
217,218,Young,-34.313499,148.301071


## Pré-Game

### Estrutura de Dados

In [3]:
class City:
    def __init__(self, id, name, lat, lng):
        self.id = id
        self.name = name
        self.lat = lat
        self.lng = lng
        self.weight = None
        self.weight_f = None
        self.roads = {}
        
    def set_weight(self, weight):
        self.weight = weight
    
        
class Road:
    def __init__(self, city, length):
        self.city = city
        self.length = length
        
def add_road_between(city1, city2):
    distance = 1.1 * sqrt((city1.lat - city2.lat)**2 + (city1.lng - city2.lng)**2)
    city1.roads[city2] = distance
    city2.roads[city1] = distance

### Preenchendo a estrutura de dados

In [4]:
cities = [None] # None é só para preencher o id = 0 (outra alternativa seria trabalhar com `índice - 1`)

# Cities
for index, row in australia.iterrows():
    city = City(row['id'], row['city'], row['lat'], row['lng'])
    cities.append(city)
    
# Roads
for city in cities:
    if city:
        if city.id % 2 == 0:
            # Road to (x - 1)
            add_road_between(city, cities[city.id - 1])
            
            # Road to (x + 2) if it exists
            if city.id + 2 < len(cities):
                add_road_between(city, cities[city.id + 2])
                
        elif city.id > 2:
            # Road to (x - 2)
            add_road_between(city, cities[city.id - 2])
            
            # Road to (x + 1) if it exists
            if city.id + 1 < len(cities):
                add_road_between(city, cities[city.id + 1])

## Algoritmo de Busca

### Parâmetros

In [5]:
# Escolher partida e destino (pode colocar o nome da cidade (formato string) ou o id (formato int))
source = 'Alice Springs'
destiny = 'Yulara'

# Também funcionaria
# source = 5
# destiny = 219

### Adiciona informação (Busca com informação)

In [6]:
# Calcula as distâncias das cidades até o destino (em linha reta)

# Encontra o nó da cidade destino
if type(destiny) == str:
    for city in cities:
        if city and city.name == destiny:
            dest_city = city
            break
        dest_city = None
else:
    for city in cities:
        if city and city.id == destiny:
            dest_city = city
            break
        dest_city = None
        

# Encontra o nó da cidade fonte
if type(source) == str:
    for city in cities:
        if city and city.name == source:
            src_city = city
            break
        src_city = None
else:
    for city in cities:
        if city and city.id == source:
            src_city = city
            break
        src_city = None

if dest_city == None or src_city == None:
    print('Cidades não encontradas')


# Para cada distância seta o peso como a distância em linha reta até a cidade destino
for city in cities:
    if city:
        city.weight = sqrt((city.lat - dest_city.lat)**2 + (city.lng - dest_city.lng)**2)

### Algoritmo

In [7]:
# A*
actual_city = src_city

# possible_path é um dicionário. Para cada cidade armazenará uma tupla (cidade_de_onde_veio, valor de f)
class PossiblePath:
    def __init__(self, parent, cost, visited):
        self.parent = parent
        self.cost = cost
        self.visited = visited
        
possible_paths = {}
possible_paths[actual_city] = PossiblePath(None, actual_city.weight, True)

while actual_city != dest_city:
    for city in actual_city.roads:
        g = city.weight
        h = actual_city.roads[city]
        # Adiciona aos caminhos possíveis uma tupla com a cidade e o custo no A*
        if city not in possible_paths:
            possible_paths[city] = PossiblePath(actual_city, g + h, False)
        elif city in possible_paths and not possible_paths[city].visited and g + h < possible_paths[city].cost:
            possible_paths[city] = PossiblePath(actual_city, g + h, False)

    # Encontra o menor candidato
    shortest_cost = None
    shortest_city = None
    for key in possible_paths:
        if not shortest_cost and not possible_paths[key].visited:
            # print('Achei', key.name)
            shortest_cost = possible_paths[key].cost
            shortest_city = key
        elif shortest_cost and possible_paths[key].cost < shortest_cost and not possible_paths[key].visited:
            shortest_cost = possible_paths[key].cost
            shortest_city = key

    # Aponta para o melhor candidato e remove ele da lista de candidatos
    actual_city = shortest_city
    possible_paths[actual_city].visited = True
actual_city.name

'Yulara'

## Resultado

In [8]:
# Percorrer a estrutura no sentido contrário para achar o caminho e somar a distância nas estradas
path = []
path_length = 0

actual_city = dest_city
while actual_city != src_city:
    path.append(actual_city)
    path_length = path_length + actual_city.roads[possible_paths[actual_city].parent]
    actual_city = possible_paths[actual_city].parent
path.append(actual_city)
    
path.reverse()
    
print('LENGTH: ', path_length)
for city in path:
    print(city.name, ' ->')

LENGTH:  1806.0673624149035
Alice Springs  ->
Ararat  ->
Armidale  ->
Ayr  ->
Ballarat  ->
Barcaldine  ->
Bathurst  ->
Bendigo  ->
Bicheno  ->
Birdsville  ->
Bordertown  ->
Bourke  ->
Boulia  ->
Bowen  ->
Broken Hill  ->
Bunbury  ->
Burketown  ->
Burnie  ->
Byron Bay  ->
Cairns  ->
Caboolture  ->
Caloundra  ->
Canberra  ->
Ceduna  ->
Charleville  ->
Clare  ->
Cobram  ->
Colac  ->
Cooma  ->
Cowra  ->
Currie  ->
Cranbourne  ->
Dalby  ->
Deniliquin  ->
Dubbo  ->
Echuca  ->
Emerald  ->
Esperance  ->
Forbes  ->
Gawler  ->
Georgetown  ->
Gingin  ->
Geraldton  ->
Gladstone  ->
Goondiwindi  ->
Griffith  ->
Gympie South  ->
Hamilton  ->
Hobart  ->
Hughenden  ->
Innisfail  ->
Ivanhoe  ->
Kalgoorlie  ->
Kalbarri  ->
Karratha  ->
Katanning  ->
Katherine  ->
Kempsey  ->
Katoomba  ->
Kiama  ->
Kimba  ->
Kingoonya  ->
Kingston South East  ->
Kwinana  ->
Laverton  ->
Leonora  ->
Longreach  ->
Manjimup  ->
Maryborough  ->
Maryborough  ->
McMinns Lagoon  ->
Meekatharra  ->
Melton  ->
Melbourne  ->
Menin